# Feature Engineering

## Imports

In [ ]:
import sys
sys.path.append("../src")

# Basic packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random as rd # generating random numbers
import datetime # manipulating date formats
import missingno as mno
# Viz
import matplotlib.pyplot as plt # basic plotting
import seaborn as sns # for prettier plots

# settings
import warnings
warnings.filterwarnings("ignore")
plt.style.use('ggplot')

In [ ]:
peptide = pd.read_csv("../data/train_peptides.csv")
protein = pd.read_csv("../data/train_proteins.csv")

## Peptide Data

### Basic FE

## Protein Data

### Basic FE